In [1]:
import pandas as pd
import numpy as np
import os
import json
import tensorflow as tf

In [ ]:
train_eur = pd.read_json('../data/train/EURUSDV1M_1w.json')
train_vix = pd.read_json('../data/train/VIX_1w.json')


In [2]:
def split_df(df):
  stock = pd.DataFrame(df['stock'].to_list(), index=df.index)
  
  df1 = df.drop(labels='stock', axis=1)
  df1 = pd.concat([stock, df1], axis=1)

  speeches = pd.DataFrame(df.iloc[:, 0].tolist(), index=df.index)
  appended_data = []
  for i in range(0, 20):
    x = pd.DataFrame(speeches.iloc[:, i].tolist(), index=speeches.index)
    appended_data.append(x)

  appended_data = pd.concat(appended_data, axis=1)
  df1 = df1.drop(labels='speech', axis=1)
  final = pd.concat([appended_data, df1], axis=1)
  return final


In [6]:
split_train_eur = split_df(train_eur)
split_train_vix = split_df(train_vix)

In [7]:
from transformers import pipeline
summarizer = pipeline('summarization')

No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [14]:
split_train_eur = split_df(train_eur)
split_train_vix = split_df(train_vix)

max_chunk = 500
cell = []
for i in range(split_train_eur.shape[0]):
    for j in range(split_train_eur.shape[1]):
        if split_train_eur.iloc[i,j] != []:
            cell = split_train_eur.iloc[i,j][0]
            cell = cell.replace('.', '.<eos>')
            cell = cell.replace('?', '?<eos>')
            cell = cell.replace('!', '!<eos>')
            sentences = cell.split('<eos>')
            current_chunk = 0 
            chunks = []
            for sentence in sentences:
                if len(chunks) == current_chunk + 1: 
                    if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                        chunks[current_chunk].extend(sentence.split(' '))
                    else:
                        current_chunk += 1
                        chunks.append(sentence.split(' '))
                else:
                    print('Summarizing; Row:', i, 'Column:', j)
                    chunks.append(sentence.split(' '))

            for chunk_id in range(len(chunks)):
                chunks[chunk_id] = ' '.join(chunks[chunk_id])
            res = summarizer(chunks, max_length=80, min_length=10, do_sample=False)
            split_train_eur.iloc[i,j] = ' '.join([summ['summary_text'] for summ in res])
        else:
            print('Empty; Row: ', i, 'Column: ', j)

Empty; Row:  0 Column:  0
Empty; Row:  0 Column:  1
Empty; Row:  0 Column:  2
Empty; Row:  0 Column:  3
Empty; Row:  0 Column:  4
Empty; Row:  0 Column:  5
Empty; Row:  0 Column:  6
Empty; Row:  0 Column:  7
Empty; Row:  0 Column:  8
Empty; Row:  0 Column:  9
Summarizing; Row: 0 Column: 10
Empty; Row:  0 Column:  11
Empty; Row:  0 Column:  12
Empty; Row:  0 Column:  13
Empty; Row:  0 Column:  14
Empty; Row:  0 Column:  15
Empty; Row:  0 Column:  16
Empty; Row:  0 Column:  17
Summarizing; Row: 0 Column: 18
Empty; Row:  0 Column:  19
Empty; Row:  0 Column:  20
Empty; Row:  0 Column:  21
Empty; Row:  0 Column:  22
Empty; Row:  0 Column:  23
Summarizing; Row: 0 Column: 24
Summarizing; Row: 0 Column: 25
Summarizing; Row: 0 Column: 26
Summarizing; Row: 0 Column: 27
Empty; Row:  0 Column:  28
Empty; Row:  0 Column:  29
Summarizing; Row: 0 Column: 30
Empty; Row:  0 Column:  31
Empty; Row:  0 Column:  32
Empty; Row:  0 Column:  33
Empty; Row:  0 Column:  34
Empty; Row:  0 Column:  35
Empty; Row

KeyboardInterrupt: 